In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_excel('Ladesaeulenkarte_Datenbankauszug31.xlsx', skiprows=5)
df.drop(['Public Key1', 'Public Key2', 'Public Key3', 'Public Key4'], axis =1, inplace = True)
df.rename(columns = {'Betreiber': 'operator',
                     'Adresse' : 'address',
                     'Postleitzahl Ort' : 'zip-code',
                     'Bundesland' : 'state',
                     'Längengrad [DG]' : 'lat',
                     'Breitengrad [DG]' : 'lon',
                     'Inbetriebnahmedatum' : 'date_of_activation',
                     'Anschlussleistung [kW]' : 'charging_power',
                     'Art der Ladeeinrichtung' : 'type_of_charger',
                     'Anzahl Ladepunkte' : 'no_of_chargingpoints',
                     'Steckertypen1' : 'plugtype_1',
                     'Steckertypen2' : 'plugtype_2',
                     'Steckertypen3' : 'plugtype_3',
                     'Steckertypen4' : 'plugtype_4',
                    },
          inplace=True,            
         )
trans_dict = {'Schnellladeeinrichtung' : 'fast charger',
             'Normalladeeinrichtung' : 'normal charger',
              'Steckdose' : 'plug',
              'Typ' : 'type',
              'Kupplung' : 'connector'
             }

df['type_of_charger'] = df['type_of_charger'].apply(lambda x : trans_dict[x])
#df[['plugtype_1', 'plugtype_2', 'plugtype_3', 'plugtype_4']] = df[['plugtype_1',
#                                                                   'plugtype_2',
#                                                                   'plugtype_3',
#                                                                   'plugtype_4']].apply(lambda x : translator(x, trans_dict))

def translator(x, trans_dict):
    x = str(x)
    if not x == np.nan:
        for k, v in trans_dict.items():
            if k in x:
                x = x.replace(k, v)
    return x

df['plugtype_1'] = df['plugtype_1'].apply(lambda x : translator(x, trans_dict))
df['plugtype_2'] = df['plugtype_2'].apply(lambda x : translator(x, trans_dict))
df['plugtype_3'] = df['plugtype_3'].apply(lambda x : translator(x, trans_dict))
df['plugtype_4'] = df['plugtype_4'].apply(lambda x : translator(x, trans_dict))

df['city'] = df['zip-code'].apply(lambda x : " ".join(x.split()[1:]))
df['zip-code'] = df['zip-code'].apply(lambda x :''.join(x.split()[:1]))

cols = df.columns.tolist()
cols = cols[0:2] + cols[-1:] + cols[2:-1]
df = df[cols]
df.to_pickle('df_ladesaeulen.pkl')


df.head(10)

In [ ]:
df['type_of_charger'].hist()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df.groupby('date_of_activation').sum())

In [ ]:
df_state = df.groupby('state').sum().sort_values(by = 'no_of_chargingpoints', ascending=False)
df_state['state'] = df_state.index.to_series()
df_state.plot('state',
              'no_of_chargingpoints',
              rot = 90,
              figsize = (14,6),
              kind ='bar',
              color =[np.random.rand(3,) for x in range(16)],
             )
#df_state.reset_index()

#barlist=plt.bar([1,2,3,4], [1,2,3,4])
#barlist[0].set_color('r')
#plt.show()